## Thai Food Classification using FastAI

- Reference: https://github.com/ai-builders/curriculum/blob/main/notebooks/01_ml_what.ipynb by Charin
- Using FastAI to fine-tune CNN model for Thai food classification

In [ ]:
# install fastbook
!pip install -q fastbook==0.0.29

In [ ]:
%%capture
!git clone https://github.com/GemmyTheGeek/FoodyDudy.git

In [ ]:
from fastbook import *

In [ ]:
food_list = [
    'green_curry', 'tepo_curry', 'liang_curry', 'taohoo_moosup', 'mara_yadsai',
    'masaman', 'orange_curry', 'cashew_chicken', 'omelette', 'sunny_side_up',
    'palo_egg', 'sil_egg', 'nun_banana', 'kua_gai', 'cabbage_fish_sauce',
    'river_prawn', 'shrimp_ob_woonsen', 'kanom_krok', 'mango_sticky_rice', 'kao_kamoo',
    'kao_klook_kapi', 'kaosoi', 'kao_pad', 'kao_pad_shrimp', 'chicken_rice',
    'kao_mok_gai', 'tom_ka_gai', 'tom_yum_kung', 'tod_mun', 'poh_pia',
    'pak_boong_fai_daeng', 'padthai', 'pad_krapao', 'pad_si_ew', 'pad_fakthong',
    'eggplant_stirfry', 'pad_hoi_lai', 'foithong', 'panaeng', 'yum_tua_ploo',
    'yum_woonsen', 'larb_moo', 'pumpkin_custard', 'sakoo_sai_moo', 'somtam',
    'moopoing','satay', 'hor_mok'
]
id2food = {i: f for i, f in enumerate(food_list)}

In [ ]:
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock), #x - image; y - single class
    get_items=get_image_files, # get all image from folders
    splitter=GrandparentSplitter(valid_name="valid"), # use parent folder as train-valid split
    get_y=parent_label, # use parent folder as label
    batch_tfms=aug_transforms(size=224)
)
data_loaders = dblock.dataloaders("FoodyDudy/images/", bs=64)

In [ ]:
# get all images
get_image_files("FoodyDudy/images/")

In [ ]:
# get y from parent label
path = get_image_files("FoodyDudy/images/")[0]
path, parent_label(path)

In [ ]:
data_loaders.train.show_batch(max_n=9,nrows=3)

In [ ]:
# data loaders, backbone: ResNet34, metric: accuracy
learn = cnn_learner(data_loaders, resnet34, metrics=accuracy)
learn.fine_tune(epochs=5, freeze_epochs=1, base_lr=2e-3)

In [ ]:
learn.show_results()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10,10))

In [ ]:
from glob import glob

test_paths = glob("FoodyDudy/images/test/15/*")

In [ ]:
pred, pred_idx, probs = learn.predict(test_paths[0])
print(pred_idx, id2food[int(pred_idx)])

## Create Gradio Application

Create Gradio Application for prediction.

In [ ]:
%%capture
!pip install gradio==3.35.0
!pip install fastapi==0.103.2

In [ ]:
import gradio as gr

def inference(img):
    """Inference function from gradio input."""
    img = PILImage.create(img)
    pred, pred_idx, probs = learn.predict(img)
    pred_sort = probs.sort(descending=True)
    predictions = {
        id2food[int(v)]: float(k)
        for k, v in zip(pred_sort.values[0:5], pred_sort.indices[0:5])
    }
    return predictions

In [ ]:
inputs = gr.inputs.Image()
outputs = gr.outputs.Label(num_top_classes=5)

interface = gr.Interface(
    fn=inference,
    inputs=inputs,
    outputs=outputs,
    title="Foody Dudy: Thai Food Prediction"
).launch(debug="True")